In [50]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import ElasticNet, ElasticNetCV, LassoCV, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, RobustScaler, scale, normalize, minmax_scale
import seaborn as sns 

In [51]:
train_city =  pd.read_csv('data/tract_health/houston_tract_health.csv')
test_city = pd.read_csv('data/tract_health/atlanta_tract_health.csv')

In [52]:
train_city

,StateAbbr,StateDesc,CountyName,CountyFIPS,TractFIPS,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,TEETHLOST_Crude95CI,Geolocation,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,LowIncomeTracts,PovertyRate,MedianFamilyIncome
0,TX,Texas,Brazoria,48039,48039663300,2661,28.7,"(25.5, 32.3)",21.2,"(20.4, 22.0)",...,"(10.1, 21.7)",POINT (-95.44262081 29.03675612),190.7,1.083186,4.052112,107.6,180.0,1,14.689781,60167.0
1,TX,Texas,Brazoria,48039,48039662600,3333,21.2,"(19.2, 23.4)",24.7,"(24.0, 25.4)",...,"( 9.9, 16.1)",POINT (-95.64407697 29.11013565),272.9,2.340433,8.416689,193.2,19.8,0,8.083076,73727.0
2,TX,Texas,Brazoria,48039,48039661100,3175,30.2,"(27.3, 33.5)",21.0,"(20.3, 21.6)",...,"(12.0, 19.3)",POINT (-95.22838051 29.44093378),224.4,1.372626,17.843796,369.4,274.8,1,13.214385,56875.0
3,TX,Texas,Brazoria,48039,48039663800,5658,31.5,"(27.3, 35.6)",19.8,"(19.0, 20.5)",...,"(11.5, 23.1)",POINT (-95.41055856 29.02960304),184.9,1.123641,4.014494,101.8,174.2,0,17.454132,65436.0
4,TX,Texas,Brazoria,48039,48039660802,7486,20.9,"(18.4, 23.5)",17.7,"(16.9, 18.4)",...,"( 6.0, 12.2)",POINT (-95.3051718 29.53971135),629.6,2.491405,7.201693,374.4,331.3,0,1.327592,109148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,TX,Texas,Harris,48201,48201231000,4282,38.4,"(34.4, 42.3)",26.2,"(25.3, 27.1)",...,"(22.0, 36.8)",POINT (-95.27707551 29.84020319),542.0,0.973493,1.646941,102.9,43.6,1,15.096045,42192.0
1057,TX,Texas,Harris,48201,48201451403,3504,19.9,"(17.5, 22.7)",13.6,"(13.0, 14.2)",...,"( 4.8, 11.3)",POINT (-95.62153667 29.74572788),103.9,0.443465,0.878847,112.3,130.1,0,10.323455,81204.0
1058,TX,Texas,Harris,48201,48201555301,960,14.1,"(11.7, 16.8)",23.2,"(21.8, 24.6)",...,"( 3.5, 10.0)",POINT (-95.51490885 30.12147692),229.2,0.848978,2.968524,330.0,142.0,0,5.231037,141964.0
1059,TX,Texas,Harris,48201,48201211500,6907,53.3,"(49.1, 57.5)",19.7,"(19.1, 20.3)",...,"(18.2, 27.8)",POINT (-95.29697741 29.76648314),292.5,0.733222,1.324845,68.5,86.0,1,27.382336,35701.0


In [53]:
test_city

,StateAbbr,StateDesc,CountyName,CountyFIPS,TractFIPS,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,TEETHLOST_Crude95CI,Geolocation,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,LowIncomeTracts,PovertyRate,MedianFamilyIncome
0,GA,Georgia,Fulton,13121,13121010204,4761,9.6,"( 7.5, 12.7)",24.8,"(23.4, 26.2)",...,"( 2.6, 8.2)",POINT (-84.41716366 33.92519129),258.6,1.051088,4.476390,103.4,103.9,0,1.821352,192813.0
1,GA,Georgia,Fulton,13121,13121005501,2307,25.2,"(21.3, 29.6)",20.0,"(18.9, 20.9)",...,"(14.5, 33.7)",POINT (-84.38372162 33.72987572),130.1,0.278257,0.821833,14.1,17.0,1,43.297645,26579.0
2,GA,Georgia,Fulton,13121,13121001202,3937,12.6,"(11.5, 14.2)",10.0,"( 9.6, 10.4)",...,"( 3.4, 6.4)",POINT (-84.38515368 33.77682317),72.2,0.620474,0.266751,37.1,0.0,0,8.592560,133177.0
3,GA,Georgia,Fulton,13121,13121011425,6906,10.8,"( 8.9, 13.5)",20.0,"(18.9, 21.3)",...,"( 2.8, 7.7)",POINT (-84.24164477 34.01149372),289.4,0.963302,3.946356,273.9,111.7,0,2.212389,186010.0
4,GA,Georgia,Fulton,13121,13121009403,4625,18.4,"(15.7, 21.9)",13.5,"(12.9, 14.1)",...,"( 5.9, 17.5)",POINT (-84.35991176 33.83156797),169.2,0.299675,1.285113,195.7,141.4,0,13.447045,68733.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,GA,Georgia,Fulton,13121,13121011623,11473,13.6,"(11.9, 15.4)",16.1,"(15.5, 16.9)",...,"( 3.9, 8.6)",POINT (-84.19032903 34.03959659),289.4,0.963302,3.946356,273.9,111.7,0,2.841046,123885.0
198,GA,Georgia,Fulton,13121,13121000600,5203,20.0,"(17.9, 22.5)",8.4,"( 8.1, 8.7)",...,"(10.3, 17.5)",POINT (-84.40584642 33.78496362),119.0,0.563022,0.387744,54.9,46.4,1,43.860730,96850.0
199,GA,Georgia,Fulton,13121,13121009603,4388,14.2,"(12.2, 17.5)",15.5,"(14.9, 16.2)",...,"( 3.5, 11.6)",POINT (-84.37581823 33.83437332),50.4,0.151147,1.358243,31.6,74.0,0,8.163265,200192.0
200,GA,Georgia,Fulton,13121,13121003900,1331,29.2,"(25.2, 33.5)",26.8,"(25.9, 27.8)",...,"(20.8, 39.1)",POINT (-84.42210552 33.75005537),115.2,0.180219,0.821719,20.4,22.3,1,36.868268,36181.0


In [54]:
name_mapping = {'access2': 'Health insurance access', 'arthritis': 'Arthritis prevalence', 'binge': 'Binge drinking prevalence',
               'bphigh': 'High blood pressure prevalence', 'bpmed': 'Medium blood pressure prevalence', 'cancer': 'Cancer prevalence',
               'casthma': 'Asthma prevalence', 'cervical': 'Cervical cancer screenings', 'chd': 'Coronary heart disease prevalence',
               'checkup': 'Routine checkups', 'cholscreen': 'Cholesterol screenings', 'colon_screen': 'Colon cancer screenings',
               'copd': 'COPD prevalence', 'corem': 'Core men\'s health', 'corew': 'Core women\'s health', 'csmoking': 'Smoking prevalence',
               'dental': 'Dental checkups', 'depression': 'Depression prevalence', 'diabetes': 'Diabetes prevalence', 'ghlth': 'General poor health prevalence',
               'highchol': 'High cholesterol prevalence', 'kidney': 'Chronic kidney disease', 'lpa': 'No physical activity', 'mammouse': 'Mammograms',
               'mhlth': 'Poor mental health prevalence', 'obesity': 'Obesity prevalence', 'phlth': 'Poor physical health', 'sleep': 'Poor sleep prevalence',
               'stroke': 'Stroke prevalence', 'teethlost': 'Teeth loss prevalence'}

results_all_nan = pd.DataFrame({'Health condition': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Food': [np.nan for c in train_city.columns if c.endswith('CrudePrev')],
                       'Physical health': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Public transport': [np.nan for c in train_city.columns if c.endswith('CrudePrev')],
                        'Education': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Houses of worship': [np.nan for c in train_city.columns if c.endswith('CrudePrev')],
                       'Poverty Rate': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'RSquared': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 
                            'MSE': [np.nan for c in train_city.columns if c.endswith('CrudePrev')]})
results_nan = pd.DataFrame({'Health condition': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Food': [np.nan for c in train_city.columns if c.endswith('CrudePrev')],
                       'Physical health': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 'Poverty Rate': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 
                            'RSquared': [np.nan for c in train_city.columns if c.endswith('CrudePrev')], 
                            'MSE': [np.nan for c in train_city.columns if c.endswith('CrudePrev')]})
add_vars = ['PovertyRate']

In [55]:
def update_all_vars():
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[0]
        results.iat[i, 2] = regr.coef_[1]
        results.iat[i, 3] = regr.coef_[2]
        results.iat[i, 4] = regr.coef_[3]
        results.iat[i, 5] = regr.coef_[4]
        results.iat[i, 6] = regr.coef_[5]
        results.iat[i, 7] = regr.score(X_test, y_test)
        results.iat[i, 8] = mse_test
def update_vars():
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[0]
        results.iat[i, 2] = regr.coef_[1]
        results.iat[i, 3] = regr.coef_[2]
        results.iat[i, 4] = regr.score(X_test, y_test)
        results.iat[i, 5] = mse_test

In [56]:
#set desert and log transformation params
all_deserts=True
log_transf = False

In [58]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
alphas = []
l1_ratios = []
i=0


for c in train_city.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = train_city[desert_measures+add_vars].to_numpy(copy=True)
        std_scaler = StandardScaler()
        scaler = MinMaxScaler()
        xscale = std_scaler.fit_transform(x)
        y = train_city[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        y = minmax_scale(y)
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(xscale, y.reshape(-1, 1))
        
        
        xtest = test_city[desert_measures+add_vars].to_numpy(copy=True)
        ytestcity = test_city[c].to_numpy(copy=True)
        ytestcity = minmax_scale(ytestcity)
        xtestscale = std_scaler.fit_transform(xtest)
        predictions = regr.predict(xtestscale)
        mse_test = mean_squared_error(ytestcity, predictions)
        
        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').